Introduction à Rust

# Séance 4 – Enum, méthodes et modules

[Pierre-Antoine Champin](http://champin.net/)

Département Info Doua – [IUT Lyon 1](http://iut.univ-lyon1.fr/)


## Enum 🦀

In [2]:
enum Mousquetaire {
    Athos,
    Porthos,
    Aramis,
}

let m1 = Mousquetaire::Athos;

In [3]:
use Mousquetaire::*;
let m2 = Porthos;

In [4]:
struct Point { x: f64, y: f64 }

enum Figure {
    Plan,
    Segment(Point, Point),
    Cercle { centre: Point, rayon: f64 },
}

In [5]:
use Figure::*;


let p1 = Point { x: 1.0, y: 2.0 };

let f1 = Segment(p1, Point { x: 1.0, y: 3.0 });


let centre = Point { x: 0.0, y: 0.0 };

let f2 = Cercle { centre, rayon: 2.5 };

### Enums et `match` déstructurant

In [6]:
fn display_figure(f: &Figure) {
    match f {
        Plan => {
            println!("le plan")
        }
        Segment(p1, p2) => {
            println!("un segment entre ({},{}) et ({},{})",
                     p1.x, p1.y, p2.x, p2.y)
        }
        Cercle { rayon, centre } => {
            println!("un cercle de centre ({},{}) et de rayon {}",
                     centre.x, centre.y, rayon);
        }
    }
}

display_figure(&f1);
display_figure(&f2);

un segment entre (1,2) et (1,3)
un cercle de centre (0,0) et de rayon 2.5


In [7]:
fn display_figure(f: &Figure) {
    match f {
        Plan => {
            println!("le plan")
        }
        Segment(p1, p2) => {
            println!("un segment entre ({},{}) et ({},{})",
                     p1.x, p1.y, p2.x, p2.y)
        }
        Cercle { rayon: r, centre: c } => {
            println!("un cercle de centre ({},{}) et de rayon {}",
                     c.x, c.y, r);
        }
    }
}

display_figure(&f1);
display_figure(&f2);

un segment entre (1,2) et (1,3)
un cercle de centre (0,0) et de rayon 2.5


### `match` déstructurant récursif

In [8]:
fn display_figure(f: &Figure) {
    match f {
        Plan => {
            println!("le plan")
        }
        Segment(Point{x, y}, Point{x: x2, y: y2}) => {
            println!("un segment entre ({},{}) et ({},{})",
                     x, y, x2, y2)
        }
        Cercle { rayon, centre: Point{x, y} } => {
            println!("un cercle de centre ({},{}) et de rayon {}",
                     x, y, rayon);
        }
    }
}

display_figure(&f1);
display_figure(&f2);

un segment entre (1,2) et (1,3)
un cercle de centre (0,0) et de rayon 2.5


### `Option<T>` et `Result<T, E>`...

... sont également des enums (génériques).

```rust
enum Option<T> {
    None,
    Some(T),
}

enum Result<T, E> {
    Ok(T),
    Err(E),
}
```

#### `match` déstructurant et `Option<T>`

In [9]:
pub fn message(o: Option<u64>) -> &'static str {
    match o {
        None => "pas de valeur",
        Some(0) => "zero",
        Some(1..=10) => "entre un et dix",
        Some(_) => "plus grand que dix",
    }
}

## Bloc `impl` et méthodes

In [10]:
fn gcd(a: u32, b: u32) -> u32 {
    fn gcd_(a: u32, b: u32) -> u32 {
        if b==0 {
            a
        } else {
            gcd_(b, a%b)
        }
    }
    if a>=b { gcd_(a, b) } else { gcd_(b, a)}
}

In [11]:
struct Fraction { num: u32, den: u32 }

impl Fraction {
    fn eval(&self) -> f64 {
        (self.num as f64)/(self.den as f64)
    }
    
    fn reduce(&mut self) {
        let gcd = gcd(self.num, self.den);
        if gcd > 1 {
            self.num /= gcd;
            self.den /= gcd;
        }
    }
}

In [12]:
let mut f = Fraction { num: 2, den: 4 };

f.reduce();
println!("{}/{} ≃ {}", f.num, f.den, f.eval());

1/2 ≃ 0.5


In [13]:
impl Fraction {
    fn add(&self, other: &Fraction) -> Fraction {
        let mut r = Fraction {
            num: self.num*other.den + other.num*self.den,
            den: self.den*other.den,
        };
        r.reduce();
        r
    }
}

In [14]:
let f1 = Fraction { num: 2, den: 4 };
let f2 = Fraction { num: 1, den: 3 };
let f3 = f1.add(&f2);
println!("{}/{} + {}/{} = {}/{}", f1.num, f1.den, f2.num, f2.den, f3.num, f3.den);


2/4 + 1/3 = 5/6


In [15]:
impl Fraction {
    fn new(num: u32, den: u32) -> Fraction {
        if den == 0 {
            panic!("Denominator cannot be zero");
        }
        Fraction { num, den }
    }
}

let f = Fraction::new(2, 4);

## Modules et visibilité

### Structure hiérarchique d'un programme

In [16]:
fn main() {
    /*...*/
}

mod toto {
    fn f1() { /*...*/ }
    
    mod tata {
        fn f2() { /*...*/ }
    }
}

mod titi {
    fn f3() { /*...*/ }
}

```rust
////// main.rs               ////// toto.rs
fn main() { /*..*/ }         fn f1() { /*...*/ }
mod toto;                    mod tata;
mod titi;

//// titi.rs                 ///// toto/tata.rs
fn f3() { /*...*/ }          fn f2() { /*...*/ }
```

### Visibilité

Les éléments suivants :

* fonction ou méthode,
* type `struct` ou `enum`,
* champ d'un `struct`,
* module,

ne sont visibles par défaut que depuis le module qui les définit (et ses sous-modules).

Pour qu'ils puissent être visibles de l'exérieur,
il faut les faire précéder du mot-clé `pub` (public).

In [17]:
pub mod tata {
    pub fn fn1(x: f64) -> f64 { x*fn2(x) }
    fn fn2(x: f64) -> f64 { if x<0.0 { -x } else { x } }
    
    pub struct Tata {
        pub c1: f64,
        c2: f64,
    }
    impl Tata {
        pub fn new(c1: f64, c2: f64) -> Tata { Tata{ c1, c2, } }
        pub fn prod(&self) -> f64 { self.c1*self.c2 }
    }
}

In [18]:
let t = tata::Tata::new(-2.0, 1.0);
tata::fn1(t.c1 + t.prod())

-16.0

In [19]:
use tata::fn1;
fn1(2.5)

6.25

In [20]:
use tata::*;
let t = Tata::new(-2.0, 1.0);
fn1(t.c1)

-4.0